- 입력 : line, current_station, next_station, arriving_time, exit
- 출력 : 가장 빠른 지하철 시간

In [14]:
import pandas as pd

In [15]:
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine

In [16]:
con1 = sqlite3.connect('subway_timetable.db')
con2 = sqlite3.connect('subway_inner_path.db') 

In [17]:
data1 = pd.read_sql_query("SELECT * FROM subway_timetable", con1)
data2 = pd.read_sql_query("SELECT * FROM subway_inner_path", con2)

In [111]:
def subway_departure_time(line, current_station, next_station, for_hour, for_minute, exit):
    # line과 current_station에 승강기 이용 가능한지 탐색
    ele = 0
    if (line in data2.line.unique()) and (current_station in data2.station.unique()):
        for i in data2[data2.line==line][data2.station == current_station].inner_path:
            if exit in i:
                ele = 1
                break
                
        # 만약 해당 출구에 승강기가 없는 경우, 승강기가 있는 출구로 반환
        if ele == 0:
            for_exit = i[-5:]
        
        # 승강장에 도착한 시간 = 도착시간 + 이동시간       
        for_time = for_minute \
                   + int(data2[data2.line==line][data2.station == current_station][data2.inner_path == i].time)
        
        _hour = int(for_time/60) + for_hour
        _minute = for_time % 60
        
        # 승강장에 도착한 시간에서 가장 빨리오는 지하철 시간 반환
        temp = data1[data1.line == line][data1.station == current_station][data1.next_station == next_station]
        hour_list = list(temp.iloc[:, 7])
        minute_list = list(temp.iloc[:, 8])
        
        for k in range(len(hour_list)):
            if hour_list[k] >= _hour:
                if minute_list[k] >= _minute:
                    break
        swtime = zeroplus(hour_list[k])+":"+zeroplus(minute_list[k])
        return for_exit, swtime
    else:
        return False

In [112]:
subway_departure_time('6호선', '상수', '광흥창', 6, 15, '2번 출구')

C:\Users\sony\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """
C:\Users\sony\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app
C:\Users\sony\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


('4번 출구', '6시 40분')

In [1]:
def subway_departure_time(line_code, current_station, next_station, for_hour, for_minute, exit):
    # line과 current_station에 승강기 이용 가능한지 탐색
    line_dic = {'1':'1호선', '2':'2호선', '3':'3호선', '4':'4호선', '5':'5호선', '6':'6호선', 
                '7':'7호선', '8':'8호선', '9':'9호선','K':'경의선/중앙선', 'B':'분당선/수인선', 
                'G':'경춘선', 'S':'신분당선', 'A':'공항철도', 'SU':'분당선/수인선'}

    ele = 0
    if (line_dic[line_code] in data2.line.unique()) and (current_station in data2.station.unique()):
        for i in data2[data2.line==line_dic[line_code]][data2.station == current_station].inner_path:
            if exit in i.split(' ↔ ')[-1]:
                ele = 1
                break
                
        # 만약 해당 출구에 승강기가 없는 경우, 승강기가 있는 출구로 반환
        if ele == 0:
            for_exit = i.split(' ↔ ')[-1]
        
        # 승강장에 도착한 시간 = 도착시간 + 이동시간
        moving_time = int(data2[data2.line==line_dic[line_code]][data2.station == current_station][data2.inner_path == i].time)
        for_time = for_minute + moving_time
        
        _hour = int(for_time/60) + for_hour
        _minute = for_time % 60
        
        # 승강장에 도착한 시간에서 가장 빨리오는 지하철 시간 반환
        temp = data1[data1.line == line_dic[line_code]][data1.station == current_station][data1.next_station == next_station]
        hour_list = list(temp.iloc[:, 7])
        minute_list = list(temp.iloc[:, 8])
        
        for k in range(len(hour_list)):
            if hour_list[k] >= _hour:
                if minute_list[k] >= _minute:
                    break
        
        if minute_list[k] < _minute:
            waiting_time = 60 + minute_list[k] - _minute
        else:
            waiting_time = minute_list[k] - _minute
            
        for_way_ = data2[data2.line==line_dic[line_code]][data2.station == current_station][data2.inner_path == i].facilities
        for l in for_way_:
            for_way = str(l).split(' ↔ ')
        for_inner_ = i.split(' ↔ ')
        
        output_path = []
        for cnt in range(len(for_inner_)-1):
            output = ""
            output += (for_inner_[cnt] +"부터 " + for_inner_[cnt+1] + "까지 " + for_way[cnt] + "로 이동") 
            output_path.append(output)
        
        swtime = zeroplus(hour_list[k])+":"+zeroplus(minute_list[k])
        return for_exit, moving_time, waiting_time, swtime, output_path
    else:
        return False

---

In [ ]:
def subway_departure_time(line_code, current_station, next_station, for_hour, for_minute, exit):
    # line과 current_station에 승강기 이용 가능한지 탐색
    line_dic = {'1':'1호선', '2':'2호선', '3':'3호선', '4':'4호선', '5':'5호선', '6':'6호선', 
                '7':'7호선', '8':'8호선', '9':'9호선','K':'경의선/중앙선', 'B':'분당선/수인선', 
                'G':'경춘선', 'S':'신분당선', 'A':'공항철도', 'SU':'분당선/수인선'}

    ele = 0
    if (line_dic[line_code] in data2.line.unique()) and (current_station in data2.station.unique()):
        for i in data2[data2.line==line_dic[line_code]][data2.station == current_station].inner_path:
            if exit in i.split(' ↔ ')[-1]:
                ele = 1
                break
                
        # 만약 해당 출구에 승강기가 없는 경우, 승강기가 있는 출구로 반환
        if ele == 0:
            for_exit = i.split(' ↔ ')[-1]
        
        # 승강장에 도착한 시간 = 도착시간 + 이동시간
        moving_time = int(data2[data2.line==line_dic[line_code]][data2.station == current_station][data2.inner_path == i].time)
        if exit==0:
            moving_time = 0
        for_time = for_minute + moving_time
        
        _hour = int(for_time/60) + for_hour
        _minute = for_time % 60
        
        # 승강장에 도착한 시간에서 가장 빨리오는 지하철 시간 반환
        temp = data1[data1.line == line_dic[line_code]][data1.station == current_station][data1.next_station == next_station]
        hour_list = list(temp.iloc[:, 7])
        minute_list = list(temp.iloc[:, 8])
        
        for k in range(len(hour_list)):
            if hour_list[k] >= _hour:
                if minute_list[k] >= _minute:
                    break
        
        if minute_list[k] < _minute:
            waiting_time = 60 + minute_list[k] - _minute
        else:
            waiting_time = minute_list[k] - _minute
            
        for_way_ = data2[data2.line==line_dic[line_code]][data2.station == current_station][data2.inner_path == i].facilities
        for l in for_way_:
            for_way = str(l).split(' ↔ ')
        for_inner_ = i.split(' ↔ ')
        
        output_path = []
        for cnt in range(len(for_inner_)-1):
            output = ""
            output += (for_inner_[cnt] +"부터 " + for_inner_[cnt+1] + "까지 " + for_way[cnt] + "로 이동") 
            output_path.append(output)
        
        swtime = zeroplus(hour_list[k])+":"+zeroplus(minute_list[k])
        return for_exit, moving_time, waiting_time, swtime, output_path
    else:
        return False